## POS tagging using modified Viterbi

### Data Preparation

In [28]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from collections import Counter

In [29]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
nltk_data[:2]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')]]

In [30]:
# splitting data into 95:5 train-test sets
train_set, test_set = train_test_split(nltk_data,test_size=0.05, random_state=100)

In [31]:
# checking size of train and test set
print(len(train_set))
print(len(test_set))

3718
196


In [32]:
# checking a sentence with tag
train_set[0]

[('One', 'NUM'),
 ('bright', 'ADJ'),
 ('sign', 'NOUN'),
 ('is', 'VERB'),
 ('that', 'ADP'),
 ('a', 'DET'),
 ('growing', 'VERB'),
 ('number', 'NOUN'),
 ('of', 'ADP'),
 ('women', 'NOUN'),
 ('have', 'VERB'),
 ('entered', 'VERB'),
 ('the', 'DET'),
 ('once', 'ADV'),
 ('male-dominated', 'ADJ'),
 ('field', 'NOUN'),
 (';', '.'),
 ('more', 'ADJ'),
 ('than', 'ADP'),
 ('a', 'DET'),
 ('third', 'ADJ'),
 ('of', 'ADP'),
 ('the', 'DET'),
 ('ringers', 'NOUN'),
 ('today', 'NOUN'),
 ('are', 'VERB'),
 ('women', 'NOUN'),
 ('.', '.')]

### Exploratory analysis on the data

In [33]:
# getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
train_tagged_words[:10]

[('One', 'NUM'),
 ('bright', 'ADJ'),
 ('sign', 'NOUN'),
 ('is', 'VERB'),
 ('that', 'ADP'),
 ('a', 'DET'),
 ('growing', 'VERB'),
 ('number', 'NOUN'),
 ('of', 'ADP'),
 ('women', 'NOUN')]

In [34]:
# getting tokens
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['One',
 'bright',
 'sign',
 'is',
 'that',
 'a',
 'growing',
 'number',
 'of',
 'women']

In [35]:
# getting unique tokens
V = list(set(tokens))
V

['victims',
 'Peabody',
 'oilman',
 'installing',
 'for',
 'attracts',
 'stated',
 'comment',
 'desultory',
 'particulars',
 'sentimental',
 'exercisable',
 'Finmeccanica',
 'Al',
 '450',
 'markets',
 'Wickliffe',
 'thrifts',
 '*-117',
 'pressure',
 'hits',
 'Westport',
 'tapes',
 '280',
 'fixtures',
 'waited',
 'securities-based',
 'Among',
 '1.49',
 'disruptive',
 'stick',
 'Died',
 'Ames',
 'intended',
 'leap',
 'functions',
 '94.2',
 'Bougainville',
 'concede',
 'opening',
 '9.3',
 '1940s',
 'Machines',
 'improvement',
 'remarks',
 'Citing',
 'hedging',
 'Hong',
 'distributes',
 'companion',
 'overtime',
 'Prime',
 'subcontractor',
 "'S",
 'manufacturers',
 'Borough',
 'brilliant',
 'Imports',
 'garage',
 'Over',
 'despise',
 'GRAINS',
 'Experts',
 'thousand',
 'mitigate',
 'marginally',
 'increasing',
 'asset-valuation',
 '77.70',
 'copying',
 '917',
 'Nature',
 'lying',
 'foreign-stock',
 'feeling',
 'ready',
 'Barnett',
 'mechanical',
 'top-selling',
 'proceeds',
 'initiative',


In [36]:
# getting unique tags
T = list(set([pair[1] for pair in train_tagged_words]))
T.sort(reverse = True)
T

['X',
 'VERB',
 'PRT',
 'PRON',
 'NUM',
 'NOUN',
 'DET',
 'CONJ',
 'ADV',
 'ADP',
 'ADJ',
 '.']

__________________________________________________________________________________________________________________

### Training HMM model using train data

In [37]:
# Creating function to calculate the emission probabilities P(Wi | Tj) 
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

In [38]:
# Creating function to calculate the transition probabilities or P(T2 | T1)
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

# creating t x t transition matrix of tags where each column is t2, each row is t1 where transition probability is P(T2 | T1)
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        
# converting the matrix to a dataframe for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,X,VERB,PRT,PRON,NUM,NOUN,DET,CONJ,ADV,ADP,ADJ,.
X,0.074433,0.204571,0.184891,0.055705,0.002857,0.062371,0.055229,0.010316,0.025393,0.144898,0.016505,0.162831
VERB,0.218438,0.168744,0.031427,0.035916,0.022448,0.111386,0.133292,0.005186,0.082050,0.091184,0.065640,0.034291
PRT,0.013123,0.405184,0.001969,0.017717,0.056102,0.245735,0.101050,0.002297,0.010171,0.019357,0.083661,0.043635
PRON,0.092720,0.484291,0.012261,0.007663,0.007280,0.211494,0.009195,0.004981,0.034100,0.023372,0.072031,0.040613
NUM,0.211824,0.016934,0.026144,0.001485,0.184195,0.352347,0.003862,0.013072,0.002674,0.035056,0.033571,0.118835
NOUN,0.028868,0.146955,0.043357,0.004721,0.009550,0.264280,0.013363,0.042921,0.016813,0.177058,0.012165,0.239951
DET,0.045323,0.040394,0.000240,0.003727,0.021640,0.637293,0.005771,0.000481,0.012623,0.009618,0.204977,0.017913
CONJ,0.008809,0.155308,0.003709,0.058414,0.042188,0.350487,0.118683,0.000464,0.053778,0.053778,0.118683,0.035698
ADV,0.023302,0.344541,0.014314,0.015646,0.031624,0.031624,0.069907,0.006991,0.077230,0.119507,0.130160,0.135153
ADP,0.034984,0.008481,0.001484,0.069119,0.061910,0.321213,0.323969,0.000848,0.013357,0.017492,0.107389,0.039754


### Building the vanilla Viterbi based POS tagger

In [39]:
# Creating function for Viterbi algorithm for tagging sequence of words of a sentence
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    T.sort(reverse=True)
    for key, word in enumerate(words):
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            # computing emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy

In [40]:
# evaluating the accuracy of the Viterbi algorithm on the test dataset

# getting list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# getting list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

# tagging the test sentences
tagged_seq = Viterbi(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy_vnila_viterbi = len(check)/len(tagged_seq)
accuracy_vnila_viterbi

0.905013750793315

In [41]:
# counting similar predicted tag that belongs to most of the unknown words
unknown_word_tag = [i for i, j in zip(tagged_seq, test_run_base) if i != j] 
unknown_tag=[]
for i in unknown_word_tag:
    unknown_tag.append(i[1])
Counter(unknown_tag).most_common()

[('X', 297),
 ('NOUN', 36),
 ('ADJ', 31),
 ('VERB', 27),
 ('ADP', 26),
 ('ADV', 12),
 ('PRT', 10),
 ('DET', 7),
 ('PRON', 2),
 ('NUM', 1)]

Most of the predicted tags are 'X' for unknown words because 'X' is the first tag in unique tag list T.

In [42]:
# counting similar actual tag that predicted incorrectly only for unknown words
actual_word_tag = [j for i, j in zip(tagged_seq, test_run_base) if i != j and i[1] == T[0]] 
actual_tag=[]
for j in actual_word_tag:
    actual_tag.append(j[1])
Counter(actual_tag).most_common()

[('NOUN', 151),
 ('VERB', 55),
 ('ADJ', 51),
 ('NUM', 34),
 ('ADV', 4),
 ('DET', 1),
 ('ADP', 1)]

Most of the actual tags that predicted incorrectly are 'NOUN'.

In [43]:
# checking unknown words with actual tag that predicted incorrectly
actual_word_tag = [j for i, j in zip(tagged_seq, test_run_base) if i != j and i[1] == T[0]] 
actual_word_tag

[('ignored', 'VERB'),
 ('Preston', 'NOUN'),
 ('Birmingham', 'NOUN'),
 ('Ala', 'NOUN'),
 ('clamped', 'VERB'),
 ('ankle', 'NOUN'),
 ('third-largest', 'ADJ'),
 ('fifth-largest', 'ADJ'),
 ('Z.', 'NOUN'),
 ('Wick', 'NOUN'),
 ('89.7', 'NUM'),
 ('141.9', 'NUM'),
 ('94.8', 'NUM'),
 ('149.9', 'NUM'),
 ('argues', 'VERB'),
 ('Sit', 'VERB'),
 ('athletic', 'ADJ'),
 ('illustrates', 'VERB'),
 ('usurp', 'VERB'),
 ('609', 'NUM'),
 ('executive-office', 'NOUN'),
 ('administer', 'VERB'),
 ('disapproved', 'VERB'),
 ('disapproval', 'NOUN'),
 ('accordance', 'NOUN'),
 ('applicable', 'ADJ'),
 ('Mead', 'NOUN'),
 ('Paper', 'NOUN'),
 ('Paper', 'NOUN'),
 ('Paper', 'NOUN'),
 ('Louisiana-Pacific', 'NOUN'),
 ('Five', 'NUM'),
 ('DeFazio', 'NOUN'),
 ('criteria', 'NOUN'),
 ('Proper', 'ADJ'),
 ('rounds', 'NOUN'),
 ('highest-pitched', 'ADJ'),
 ('descending', 'VERB'),
 ('62-year-old', 'ADJ'),
 ('forest-product', 'NOUN'),
 ('unsolicited', 'ADJ'),
 ('3.19', 'NUM'),
 ('impressive', 'ADJ'),
 ('29year', 'ADJ'),
 ('nine-month', 

As we can see above that:
- Unknown words such as names of person, places etc. for example 'Wick', 'Preston', 'Birmingham' etc should be tagged as 'NOUN'.
- Unknown words such as words having suffix(ed, ing etc.), for example 'ignored', 'clamped', 'disapproved', 'wrestling', 'maturing' etc should be tagged as 'VERB'.
- Unknown words such as words having suffix(est, er, ive, ent etc.), for example 'third-largest', 'fifth-largest', 'Proper', 'impressive',  'intelligent' etc should be tagged as 'ADJ'.
- Unknown words such as numbers, for example '89.7', '141.9', '94.8', '692' etc should be tagged as 'NUM'.
- Unknown words such as words having suffix(ly etc.), for example 'marvelously', 'faultlessly', 'broadly' etc should be tagged as 'ADV'.

In [44]:
# incorrect tagged word with correct tagged word, previous correct tagged word and next correct tagged word
# [(previous correct tagged word), ((incorrect tagged word), (correct tagged word)), (previous correct tagged word)]
incorrect_tagged_cases = [[test_run_base[i-1],j,test_run_base[i+1]] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1] and j[0][1] == T[0]]
incorrect_tagged_cases 

[[('apparently', 'ADV'),
  (('ignored', 'X'), ('ignored', 'VERB')),
  ('reports', 'NOUN')],
 [('.', '.'), (('Preston', 'X'), ('Preston', 'NOUN')), ('G.', 'NOUN')],
 [('Foster', 'NOUN'),
  (('Birmingham', 'X'), ('Birmingham', 'NOUN')),
  (',', '.')],
 [(',', '.'), (('Ala', 'X'), ('Ala', 'NOUN')), ('.', '.')],
 [('has', 'VERB'), (('clamped', 'X'), ('clamped', 'VERB')), ('on', 'ADP')],
 [('their', 'PRON'), (('ankle', 'X'), ('ankle', 'NOUN')), ('like', 'ADP')],
 [('the', 'DET'),
  (('third-largest', 'X'), ('third-largest', 'ADJ')),
  ('producer', 'NOUN')],
 [('the', 'DET'),
  (('fifth-largest', 'X'), ('fifth-largest', 'ADJ')),
  ('exporter', 'NOUN')],
 [('Charles', 'NOUN'), (('Z.', 'X'), ('Z.', 'NOUN')), ('Wick', 'NOUN')],
 [('Z.', 'NOUN'), (('Wick', 'X'), ('Wick', 'NOUN')), ('.', '.')],
 [('#', '.'), (('89.7', 'X'), ('89.7', 'NUM')), ('million', 'NUM')],
 [('$', '.'), (('141.9', 'X'), ('141.9', 'NUM')), ('million', 'NUM')],
 [('#', '.'), (('94.8', 'X'), ('94.8', 'NUM')), ('million', 'NUM'

In [45]:
# counting similar predicted tag sets that belongs to unknown words with previous word and next word
incorrect_tagged_cases = [(tagged_seq[i-1][1],j[0][1],tagged_seq[i+1][1]) for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1] and j[0][1] == T[0]]
Counter(incorrect_tagged_cases).most_common(15)

[(('.', 'X', 'NOUN'), 21),
 (('DET', 'X', 'NOUN'), 21),
 (('NOUN', 'X', '.'), 14),
 (('.', 'X', 'NUM'), 12),
 (('.', 'X', 'X'), 12),
 (('ADP', 'X', 'NOUN'), 10),
 (('.', 'X', '.'), 9),
 (('X', 'X', 'NOUN'), 9),
 (('X', 'X', '.'), 8),
 (('ADP', 'X', '.'), 8),
 (('X', 'X', 'X'), 8),
 (('DET', 'X', 'ADP'), 8),
 (('NOUN', 'X', 'NOUN'), 7),
 (('VERB', 'X', 'X'), 6),
 (('ADJ', 'X', '.'), 6)]

In [46]:
# counting similar actual tag sets that belongs to unknown words with previous word and next word
incorrect_tagged_cases = [(test_run_base[i-1][1],test_run_base[i][1], test_run_base[i+1][1]) for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1] and j[0][1] == T[0]]
Counter(incorrect_tagged_cases).most_common(15)

[(('NOUN', 'NOUN', '.'), 20),
 (('.', 'NOUN', 'NOUN'), 16),
 (('NOUN', 'NOUN', 'NOUN'), 12),
 (('.', 'NUM', 'NUM'), 12),
 (('DET', 'ADJ', 'NOUN'), 10),
 (('DET', 'NOUN', 'NOUN'), 10),
 (('.', 'ADJ', 'NOUN'), 9),
 (('.', 'NOUN', '.'), 8),
 (('DET', 'NOUN', 'ADP'), 8),
 (('ADJ', 'NOUN', '.'), 7),
 (('ADJ', 'NOUN', 'ADP'), 6),
 (('ADP', 'NOUN', '.'), 6),
 (('X', 'VERB', 'NOUN'), 5),
 (('VERB', 'VERB', 'X'), 5),
 (('.', 'NUM', 'X'), 5)]

As we can see in the above two results of actual tag sets and predicted tag sets of unknown words that tag sets are following some pattern with previous and next tags. 

For example: actual tag sets ('.', 'NUM', 'NUM') and predicted tag set ('.', 'X', 'NUM'); if previous tag of unknown word is '.' and next tag of unknown word is 'NUM' then unknown word should be tagged as 'NUM', not as 'X'.

__________________________________________________________________________________________________________________

### Solve the problem of unknown words

### Technique I
Previous and next tag based Technique

In [47]:
# Modifing Viterbi algorithm for tagging sequence of words of a sentence
def Viterbi_modify1(words, train_bag = train_tagged_words):
    state = []
    final_words = words.copy()
    T = list(set([pair[1] for pair in train_bag]))
    T.sort(reverse=True)
    for key, word in enumerate(words):
        p = [] 
        word_presence = 0
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            # computing emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            word_presence += word_given_tag(words[key], tag)[0] 
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        # if word is unknown then word_presence will be 0
        if word_presence == 0:
            words[key] = "ABCD" # replacing unknown words with ABCD
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    t_w = list(zip(words, state))
    count_t_w = len(t_w)
    # loop for tagging unknown words
    for index in range(count_t_w):
        tag_max = ""
        if t_w[index][0]=='ABCD':
            if index == 0:
                t1 = '.' # for the first word, previous tag should be '.'
            else:
                t1 = t_w[index-1][1]
            if index == (count_t_w-1):
                t3 = '.' # for the last word, next tag should be '.'
            else:    
                t3 = t_w[index+1][1]
            tags = [pair[1] for pair in train_tagged_words]
            count_t3_t1 = 0
            # counting the combinations of previous and next tag of unknown word
            for i in range(len(tags)-2):
                if tags[i]==t1 and tags[i+2] == t3:
                    count_t3_t1 += 1
            prob = []
            # counting the combination of previous and next tag with all possible tags of unknown word
            for t2 in T:
                count_t3_t2_t1 = 0
                for i in range(len(tags)-2):
                    if tags[i]==t1 and tags[i+1] == t2 and tags[i+2] == t3:
                        count_t3_t2_t1 += 1
                prob.append(count_t3_t2_t1/count_t3_t1)
            probmax = max(prob)
            # getting state for which probability is maximum
            tag_max = T[prob.index(probmax)]
            state[index] = tag_max

    return list(zip(final_words, state))

#### Evaluating tagging accuracy

In [48]:
# evaluating the accuracy of the Viterbi algorithm on the test dataset

# getting list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# getting list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

# tagging the test sentences
tagged_seq = Viterbi_modify1(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy_modfy1_viterbi = len(check)/len(tagged_seq)
accuracy_modfy1_viterbi

0.9314575840913899

#### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [49]:
print("Tagging accuracy of vanilla Viterbi algorithm: {}%".format(int(round(accuracy_vnila_viterbi, 2)*100)))
print("Tagging accuracy of modified Viterbi algorithm: {}%".format(int(round(accuracy_modfy1_viterbi, 2)*100)))

Tagging accuracy of vanilla Viterbi algorithm: 91%
Tagging accuracy of modified Viterbi algorithm: 93%


Tagging accuracy improved by 2% with modified Viterbi algorithm.

#### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [50]:
# Three sentences from sample test file which is incorrectly tagging by original POS tagger(Viterbi())
# and correctly tagging by modified POS tagger
sent1 = "Before entering politics, Donald Trump was a domineering businessman and a television personality."
sent2 = "Android is a mobile operating system developed by Google."
sent3 = "NASA invited social media users to experience the launch of ICESAT-2 Satellite."
print("1) Incorrectly tagged sentence:")
print(Viterbi(word_tokenize(sent1)))
print("1) Correctly tagged sentence:")
print(Viterbi_modify1(word_tokenize(sent1)))
print("\n")
print("2) Incorrectly tagged sentence:")
print(Viterbi(word_tokenize(sent2)))
print("2) Correctly tagged sentence:")
print(Viterbi_modify1(word_tokenize(sent2)))
print("\n")
print("3) Incorrectly tagged sentence:")
print(Viterbi(word_tokenize(sent3)))
print("3) Correctly tagged sentence:")
print(Viterbi_modify1(word_tokenize(sent3)))

1) Incorrectly tagged sentence:
[('Before', 'ADP'), ('entering', 'VERB'), ('politics', 'NOUN'), (',', '.'), ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('domineering', 'X'), ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), ('personality', 'X'), ('.', '.')]
1) Correctly tagged sentence:
[('Before', 'ADP'), ('entering', 'VERB'), ('politics', 'NOUN'), (',', '.'), ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('domineering', 'ADJ'), ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), ('personality', 'NOUN'), ('.', '.')]


2) Incorrectly tagged sentence:
[('Android', 'X'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'X'), ('.', '.')]
2) Correctly tagged sentence:
[('Android', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed',

As we can see that modified viterbi algorithm correctly tagged the words:
- In the first sentence, algorithm changed ('personality', 'X') to ('personality', 'NOUN').
- In the second sentence, algorithm changed ('Google', 'X') to ('Google', 'NOUN').
- In the third sentence, algorithm changed ('NASA', 'X') to ('NASA', 'NOUN'), ('ICESAT-2', 'X') to ('ICESAT-2', 'NOUN'), ('Satellite', 'X') to ('Satellite', 'NOUN').

__________________________________________________________________________________________________________________

__________________________________________________________________________________________________________________

### Technique II
Morphology based Technique

In [51]:
# Modifing Viterbi algorithm 2 for tagging sequence of words of a sentence
def Viterbi_modify2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    T.sort(reverse=True)
    for key, word in enumerate(words):
        p = [] 
        word_presence = 0
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
            # computing emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            word_presence += word_given_tag(words[key], tag)[0] 
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        # if word is unknown then word_presence will be 0
        if word_presence == 0:
            # regular expressions based condition for unknown word that has certain prefix or suffix
            if type(re.search(r'^(\d)+(.)?(\d)+$', words[key])) == re.Match: 
                state.append('NUM')
            elif type(re.search(r'^[a-z-]{2,}(ed|ing|ize)$', words[key])) == re.Match:
                state.append('VERB')
            elif type(re.search(r'^[a-z-]{1,}(est|ous|er|en|ful|ive|ent|ble)$', words[key])) == re.Match:
                state.append('ADJ')
            elif type(re.search(r'^(un)[a-z-]{1,}', words[key])) == re.Match:
                state.append('ADJ')
            elif type(re.search(r'[a-z-]{2,}ly$', words[key])) == re.Match:
                state.append('ADV')
            else:
                state.append('NOUN')
        else:
            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy

In [52]:
# evaluating the accuracy of the Viterbi algorithm on the test dataset

# getting list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# getting list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

# tagging the test sentences
tagged_seq = Viterbi_modify2(test_tagged_words)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy_modfy2_viterbi = len(check)/len(tagged_seq)
accuracy_modfy2_viterbi

0.9496509414004655

#### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [53]:
print("Tagging accuracy of vanilla Viterbi algorithm: {}%".format(int(round(accuracy_vnila_viterbi, 2)*100))) 
print("Tagging accuracy of modified Viterbi algorithm: {}%".format(int(round(accuracy_modfy2_viterbi, 2)*100)))

Tagging accuracy of vanilla Viterbi algorithm: 91%
Tagging accuracy of modified Viterbi algorithm: 95%


Tagging accuracy improved by 4% with modified Viterbi algorithm.

#### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [54]:
# Three sentences from sample test file which is incorrectly tagging by original POS tagger(Viterbi())
# and correctly tagging by modified POS tagger
sent1 = "Android is a mobile operating system developed by Google."
sent2 = "Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013."
sent3 = "This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe."
print("1) Incorrectly tagged sentence:")
print(Viterbi(word_tokenize(sent1)))
print("1) Correctly tagged sentence:")
print(Viterbi_modify2(word_tokenize(sent1)))
print("\n")
print("2) Incorrectly tagged sentence:")
print(Viterbi(word_tokenize(sent2)))
print("2) Correctly tagged sentence:")
print(Viterbi_modify2(word_tokenize(sent2)))
print("\n")
print("3) Incorrectly tagged sentence:")
print(Viterbi(word_tokenize(sent3)))
print("3) Correctly tagged sentence:")
print(Viterbi_modify2(word_tokenize(sent3)))

1) Incorrectly tagged sentence:
[('Android', 'X'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'X'), ('.', '.')]
1) Correctly tagged sentence:
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.')]


2) Incorrectly tagged sentence:
[('Android', 'X'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'X'), ('worldwide', 'X'), ('on', 'ADP'), ('smartphones', 'X'), ('since', 'ADP'), ('2011', 'X'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'X'), ('.', '.')]
2) Correctly tagged sentence:
[('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('an

As we can see that modified viterbi algorithm correctly tagged the words:
- In the first sentence, algorithm changed ('Android', 'X') to ('Android', 'NOUN'), ('Google', 'X') to ('Google', 'NOUN').
- In the second sentence, algorithm changed ('Android', 'X') to ('Android', 'NOUN'), ('OS', 'X') to ('OS', 'NOUN'), ('worldwide', 'X') to ('worldwide', 'NOUN'), ('smartphones', 'X') to ('smartphones', 'NOUN'), ('2011', 'X') to ('2011', 'NUM'), ('2013', 'X') to ('2013', 'NUM').
- In the third sentence, algorithm changed (('Cup', 'X') to ('Cup', 'NOUN').